In [2]:
import dask.dataframe as dd
import pandas as pd
from huggingface_hub import hf_hub_download
import tqdm as notebook_tqdm

# Исходные файлы загружены на ресурс https://huggingface.co/datasets
REPO_ID = "DmitriiMiptEdu/event_data"
adv = 'adv_data.csv'
event = 'event_data.csv'
users = 'users_data.csv'


In [3]:
# Импорт данных из CSV файлов
event_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=event, repo_type="dataset"))
adv_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=adv, repo_type="dataset"))
users_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=users, repo_type="dataset"))

In [17]:
event_data.columns['user_phone']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [4]:
# Преобразование полей датасетов во временные метки
event_data['event_dtm'] = dd.to_datetime(event_data['event_dtm'])
event_data['update_dtm'] = dd.to_datetime(event_data['update_dtm'])
adv_data['campaign_start_dtm'] = dd.to_datetime(adv_data['campaign_start_dtm'])
adv_data['campaign_end_dtm'] = dd.to_datetime(adv_data['campaign_end_dtm'])
users_data['registration_dtm'] = dd.to_datetime(users_data['registration_dtm'])
users_data['delete_dtm'] = dd.to_datetime(users_data['delete_dtm'])


In [7]:
event_data.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, id to product_type
dtypes: datetime64[ns](2), object(4), float64(2), int64(1)

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [8]:
# Объединение users_data и event_data
merged_data = dd.merge(event_data, users_data, on='user_phone', how='left')

In [10]:
merged_data.head(10)

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+--------+----------+
| Column       | Found  | Expected |
+--------------+--------+----------+
| event_status | object | float64  |
| user_phone   | object | int64    |
+--------------+--------+----------+

The following columns also raised exceptions on conversion:

- event_status
  ValueError("could not convert string to float: 'deal'")
- user_phone
  ValueError("invalid literal for int() with base 10: '+791xxxxxxx1'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'event_status': 'object',
       'user_phone': 'object'}

to the call to `read_csv`/`read_table`.

In [5]:

# Объединение с adv_data
merged_data = dd.merge(merged_data, adv_data, on='user_id', how='left')

In [6]:
# Анализ событий пользователей
# Сортировка событий по пользователям и времени
merged_data = merged_data.sort_values(by=['user_phone', 'event_dtm'])

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+------------+--------+----------+
| Column     | Found  | Expected |
+------------+--------+----------+
| delete_dtm | object | float64  |
| user_phone | object | int64    |
+------------+--------+----------+

The following columns also raised exceptions on conversion:

- delete_dtm
  ValueError("could not convert string to float: '2024-02-26 20:55:17.844081'")
- user_phone
  ValueError("invalid literal for int() with base 10: '+791xxxxxxx1'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'delete_dtm': 'object',
       'user_phone': 'object'}

to the call to `read_csv`/`read_table`.

In [ ]:
# Идентификация конверсий
conversions = merged_data[merged_data['event_status'] == 'success']

In [ ]:
# Разработка модели атрибуции

# Разработка модели атрибуции

In [ ]:
# Реализация модели линейной атрибуции
def linear_attribution(events):
    n = len(events)
    attribution = 1.0 / n
    events['attribution'] = attribution
    return events

# Применение линейной модели атрибуции
attributed_data = merged_data.groupby('user_phone').apply(linear_attribution, meta=merged_data)


In [ ]:
# Визуализация результатов
import matplotlib.pyplot as plt

# Сбор данных для визуализации
source_attribution_pd = source_attribution.compute()

# Пример простой визуализации распределения атрибуции
source_attribution_pd.plot(kind='bar')
plt.title('Атрибуция по источникам трафика')
plt.xlabel('Источник')
plt.ylabel('Атрибуция')
plt.show()